# Setup

In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import imutils

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

### Question 1:
Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)
- img is the source image
- num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
- line_color is the color of the grid line.
- The output of the function should be image with grids

In [2]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    img = cv.imread(img)
    img_copy = img.copy()

    height, width = img.shape[:2]
    num_horizontal_patches = num_horizontal_grid
    num_vertical_patches = num_vertical_grid

    # M and N are basically number of pixels per patch
    M, N = int(height/num_horizontal_patches), int(width/num_vertical_patches)

    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
    #         if (height - y) < M or (width - x) < N:
    #             break

            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N

            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
                cv.imshow('tile', tile)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

    cv.imshow('patched image', img_copy)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
crop_grid('dog.jfif', 3, 3, (0,255,0))

### Question 2:
Display image sequences of smooth transition of two images with different values of α. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.

In [4]:
img1 = cv.imread('lena.jpg')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

# array of values of alpha
alpha = np.arange(0, 1.1, 0.10)

for i in (alpha):
    dst = cv.addWeighted(img1, i, img2, 1-i, 0)
    cv.imshow('blend_image', dst)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Question 3:
Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix

In [5]:
img = cv.imread('lena.jfif')
rows, cols = img.shape[:2]

dst = imutils.rotate_bound(img, 45)

cv.imshow('rotated_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

### Question 4:
Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. The result should resemble the following:

In [6]:
img1 = cv.imread('flower.jfif')
img2 = cv.imread('native-bee.png')

# using libraries
# copy (with modifications) from: https://docs.opencv.org/3.4/d0/d86/tutorial_py_image_arithmetics.html

# # I want to put flower on top-left corner, So I create a ROI
# rows,cols,channels = img1.shape
# roi = img2[0:rows, 0:cols]

# # Now create a mask of flower and create its inverse mask also
# img1gray = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
# ret, mask = cv.threshold(img1gray, 70, 255, cv.THRESH_BINARY)
# mask_inv = cv.bitwise_not(mask)

# # Now black-out the area of logo in ROI
# img2_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# # Take only region of logo from logo image.
# img1_fg = cv.bitwise_and(img1,img1,mask = mask)

# # Put logo in ROI and modify the main image
# dst = cv.add(img2_bg,img1_fg)
# img2[0:rows, 0:cols ] = dst

# cv.imshow('image2', img2)
# cv.waitKey(0)
# cv.destroyAllWindows()

# writting own code is always faster and easier...
for i in range (img1.shape[0]):
    for j in range (img1.shape[1]):
        if img1[i,j].mean() >= 70:
            img2[i,j] = img1[i,j]

cv.imshow('image2', img2)
cv.waitKey(0)
cv.destroyAllWindows()